In [16]:
from nomic import atlas
import numpy as np
import pickle
import os
import openai

from dotenv import load_dotenv
load_dotenv()
API_KEY = os.getenv('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = API_KEY
openai.api_key = API_KEY

In [14]:
cs_data = None
with open("../../data/dev/cs_data.pkl", "rb") as f:
    cs_data = pickle.load(f)

In [24]:
embeddings = []
for doc_tuple in cs_data:
    passage = doc_tuple[0].strip()
    embedding = openai.Embedding.create(input=passage, model="text-embedding-ada-002")
    embeddings.append(embedding)

APIError: The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID b6f07c12bcebf480a84fa58ca3f1ad22 in your message.) {
  "error": {
    "message": "The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID b6f07c12bcebf480a84fa58ca3f1ad22 in your message.)",
    "type": "server_error",
    "param": null,
    "code": null
  }
}
 500 {'error': {'message': 'The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID b6f07c12bcebf480a84fa58ca3f1ad22 in your message.)', 'type': 'server_error', 'param': None, 'code': None}} {'Date': 'Thu,

In [28]:
embeddings = [x["data"][0]["embedding"] for x in embeddings]

In [29]:
project = atlas.map_embeddings(
    embeddings=np.array(embeddings)
)

2023-05-17 21:03:14.606 | INFO     | nomic.project:_create_project:965 - Creating project `innocent-patriot` in organization `mmb4`
2023-05-17 21:03:15.859 | INFO     | nomic.atlas:map_embeddings:100 - Uploading embeddings to Atlas.
1it [00:01,  1.23s/it]
2023-05-17 21:03:17.111 | INFO     | nomic.project:_add_data:1577 - Upload succeeded.
2023-05-17 21:03:17.113 | INFO     | nomic.atlas:map_embeddings:119 - Embedding upload succeeded.
2023-05-17 21:03:18.343 | INFO     | nomic.project:create_index:1282 - Created map `innocent-patriot` in project `innocent-patriot`: https://atlas.nomic.ai/map/6c6c17d3-ac2f-4240-9458-d265f75b588b/6a721206-647a-4a4a-aef9-49717e3aa262
2023-05-17 21:03:18.345 | INFO     | nomic.atlas:map_embeddings:132 - innocent-patriot: https://atlas.nomic.ai/map/6c6c17d3-ac2f-4240-9458-d265f75b588b/6a721206-647a-4a4a-aef9-49717e3aa262


In [27]:
# with open("../data/embeddings.pkl", "wb") as f:
#     pickle.dump(embeddings, f)

In [35]:
documents = [{"text": content.strip(), "id": idx} for idx, (content, source) in enumerate(cs_data)]

In [36]:
len(documents)

276

In [37]:
project = atlas.map_text(data=documents,
                         id_field='id',
                         indexed_field='text',
                         name='CS Data',
                         description='CS Data for Labeling'
                         )

2023-05-17 21:15:39.963 | INFO     | nomic.project:_create_project:965 - Creating project `CS Data` in organization `mmb4`
2023-05-17 21:15:41.487 | INFO     | nomic.atlas:map_text:212 - Uploading text to Atlas.
2023-05-17 21:15:41.505 | WARNING  | nomic.project:_validate_and_correct_arrow_upload:244 - id_field is not a string. Converting to string from int32
1it [00:00,  1.12it/s]
2023-05-17 21:15:42.415 | INFO     | nomic.project:_add_data:1577 - Upload succeeded.
2023-05-17 21:15:42.417 | INFO     | nomic.atlas:map_text:228 - Text upload succeeded.
2023-05-17 21:15:44.294 | INFO     | nomic.project:create_index:1282 - Created map `CS Data` in project `CS Data`: https://atlas.nomic.ai/map/97a0d075-7ebb-4502-91e8-0145291df751/a9fa0fa1-8940-4534-9ac4-0aca686c4ba5
2023-05-17 21:15:44.296 | INFO     | nomic.atlas:map_text:242 - CS Data: https://atlas.nomic.ai/map/97a0d075-7ebb-4502-91e8-0145291df751/a9fa0fa1-8940-4534-9ac4-0aca686c4ba5


In [46]:
from openai.embeddings_utils import cosine_similarity

In [39]:
to_embed = {
    "cs_331_passage_with_title": "CSCI 331 (Introduction To Computer Security) Section 01 year: 2024, semester: Fall, sectionType: in-person, classType: Lecture, instructors: [{'name': 'Daniel W. Barowy'}], meetings: [{'days': 'MR', 'start': '14:35', 'end': '15:50', 'facility': ''}]",
    "cs_331_passage_no_title": "CSCI 331 Section 01 year: 2024, semester: Fall, sectionType: in-person, classType: Lecture, instructors: [{'name': 'Daniel W. Barowy'}], meetings: [{'days': 'MR', 'start': '14:35', 'end': '15:50', 'facility': ''}]",
    "cs_331": "CS 331",
    "intro_to_comp_sec": "Intro to Computer Security"
}

In [40]:
embeddings = {}

In [41]:
for k, v in to_embed.items():
    embeddings[k] = openai.Embedding.create(
    input=v, model="text-embedding-ada-002"
)["data"][0]["embedding"]

In [48]:
import itertools
for k1, k2 in itertools.product(embeddings.keys(), embeddings.keys()):
    if k1 != k2:
        print(f"Similarity {k1} <> {k2} = {cosine_similarity(embeddings[k1], embeddings[k2])}")

Similarity cs_331_passage_with_title <> cs_331_passage_no_title = 0.9687680201077175
Similarity cs_331_passage_with_title <> cs_331 = 0.848893386392108
Similarity cs_331_passage_with_title <> intro_to_comp_sec = 0.8538556263664516
Similarity cs_331_passage_no_title <> cs_331_passage_with_title = 0.9687680201077175
Similarity cs_331_passage_no_title <> cs_331 = 0.8482636838466528
Similarity cs_331_passage_no_title <> intro_to_comp_sec = 0.771786030426771
Similarity cs_331 <> cs_331_passage_with_title = 0.848893386392108
Similarity cs_331 <> cs_331_passage_no_title = 0.8482636838466528
Similarity cs_331 <> intro_to_comp_sec = 0.8027270150940974
Similarity intro_to_comp_sec <> cs_331_passage_with_title = 0.8538556263664516
Similarity intro_to_comp_sec <> cs_331_passage_no_title = 0.771786030426771
Similarity intro_to_comp_sec <> cs_331 = 0.8027270150940974


Similarity cs_331_passage_with_title <> cs_331 = 0.848893386392108  
Similarity cs_331_passage_with_title <> intro_to_comp_sec = 0.8538556263664516  
Similarity cs_331_passage_no_title <> cs_331 = 0.8482636838466528  
Similarity cs_331_passage_no_title <> intro_to_comp_sec = 0.771786030426771  

## "Intro to Computer Security" is always higher similarity than "CS 331", and there's a >8 percentage point increase in similarity when searching for "Intro to Computer Security" with a title compared to without a title